In [1]:
from simulation import *
from aesthetics import *
from models import *
from data import *
from analysis import *

import glob
import os as os
import re as re
import pandas as pd

%reload_ext autoreload
%autoreload 2
%matplotlib inline
mpl.rc('text', usetex=True)

In [2]:
calculation = False
df = pd.DataFrame()
if calculation:
    for concentration in np.arange(-6, 0, 0.1):
        for name in names:
            directional_flux, intersurface_flux, driven_flux = summarize_fluxes(name, concentration=10**concentration,
                                                                               data_source='pka_md_data') 
            if directional_flux < 0:
                negative=True
            else:
                negative=False
            max_power, max_load = summarize_power_and_load(name, concentration=10**concentration, negative=negative, 
                                                           debug=False, data_source='pka_md_data')
            df = df.append(pd.DataFrame({'Concentration': concentration,
                                    'Directional flux': directional_flux,
                                    'Intersurface flux': intersurface_flux,
                                    'Driven flux': driven_flux,
                                    'File': name,
                                    'Max power': max_power,
                                    'Max load' : max_load,
                                    'ResID': re.match('.*?([0-9]+)$', name).group(1),
                                    }, index=[0]), ignore_index=True)

    df.to_pickle('pka-concentration-scan.pickle')
else:
    df = pd.read_pickle('pka-concentration-scan.pickle')

In [6]:
def return_concentration_slice(df, concentration):
    tmp = df[np.round(df['Concentration'], 1) ==  np.round(concentration, 1)]
    return tmp

In [9]:
tmp = return_concentration_slice(df, -0.1)

In [10]:
tmp['Max power'].sum()

59.44412721822106

In [21]:
tmp.sort_values('Max load', ascending=False).head()

,Concentration,Directional flux,Driven flux,File,Intersurface flux,Max load,Max power,ResID
79092,-0.1,28.610131,40.957264,chi1ASP152,7.949038,0.40099,5.538951,152
79372,-0.1,15.239268,136.418205,chi2ASN157,17.819112,0.34099,2.585261,157
79994,-0.1,7.448218,75.327080,phiS2P324,15.187449,0.18099,0.665612,324
79568,-0.1,13.128531,29.545781,chi2TRP16,4.915500,0.17099,1.123899,16
80341,-0.1,42.741092,87.101455,psiSER311,13.940469,0.15099,3.167703,311


How many motors show significant amount of load bearing at this concentration?

In [22]:
sum(tmp['Max load'] > 0.01) + sum(tmp['Max load'] < -0.01)

35

And how many of those show moderate directional flux?

In [24]:
sum(tmp['Directional flux'] > 1) + sum(tmp['Directional flux'] < -1)

136

In [25]:
delta_G = -7.2 # kcal per mol at [S] = 0.1 M
this = simulation(data_source = 'pka_md_data')
this.name = 'chi1ASP152'
this.cSubstrate = 0.1
this.simulate()
prob_bound_state = np.sum(this.ss[this.bins:2*this.bins])
power_in = delta_G * this.catalytic_rate * prob_bound_state

In [27]:
print(power_in)

-1001.59142773


What is the maximum power generated by any angle at [S] = 1e-3 M?

In [28]:
tmp = return_concentration_slice(df, -3)

In [34]:
tmp.sort('Max power', ascending=False).head()

/home/dslochower/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Concentration,Directional flux,Driven flux,File,Intersurface flux,Max load,Max power,ResID
40567,-3.0,-40.261997,77.772012,chi2GLU194,18.562073,-0.09799,2.016239,194
40272,-3.0,-19.758834,81.945888,chi1GLU194,20.036930,-0.09299,0.911775,194
40232,-3.0,19.571276,27.885801,chi1ASP152,5.444010,0.08799,0.850275,152
41521,-3.0,-15.960652,59.706824,psiVAL109,10.063211,-0.05599,0.447352,109
40392,-3.0,-5.850426,54.425885,chi1PHE140,14.212754,-0.13099,0.396225,140


In [67]:
percent = 100 * concentration_counts[np.round(concentration_counts['Concentration'], 1) ==  np.round(-3.0, 1)]['Directional flux > 1'].values[0] / len(names)

In [68]:
percent

8.7313432835820901

In [50]:
df = pd.read_pickle('adk-concentration-counts.pickle')

In [55]:
calculation = True
adk_concentration_counts = pd.DataFrame()
if calculation:

    for concentration in np.arange(-6, 0, 0.1):
        # This is the dataframe that matches this concentration...
        tmp = df[np.round(df['Concentration'], 1) ==  np.round(concentration, 1)]
        tmp['Directional flux'] = tmp['Directional flux'].abs()
        tmp['Driven flux'] = tmp['Driven flux'].abs()

        adk_concentration_counts = adk_concentration_counts.append(pd.DataFrame({
                                'Concentration' : concentration,
                                'Directional flux > 1' : np.sum(tmp['Directional flux'] > 1),
                                'Driven flux > 1' : np.sum(tmp['Driven flux'] > 1),
                    }, index=[0]), ignore_index=True)
    adk_concentration_counts.to_pickle('adk-concentration-counts.pickle')
else:
    adk_concentration_counts = pd.read_pickle('adk-concentration-counts.pickle')

/home/dslochower/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dslochower/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
adk_percent = 100 * adk_concentration_counts[np.round(adk_concentration_counts['Concentration'], 1) ==  np.round(-3.0, 1)]['Directional flux > 1'].values[0] / 903

In [57]:
adk_percent

13.732004429678849